# Capacitated Vehicle Routing Problem with Gmaps
Pada sesi ini kita akan menyelesaikan kasus fiktif Capacitated VRP menggunakan bantuan pulp untuk optimasi serta gmaps untuk visualisasi dan routing. Saya menggunakan data lokasi sebuah perusahaan retail sebagai gambaran penerapan. Akan dilakukan pengiriman produk X dari Alfamidi (Depot) ke enam Alfamart. Diasumsikan truk yang akan mengangkut produk X memiliki kapasitas 100 unit dan terdapat perbedaan demand yang jelas antar retailer.


Pada model capacitated VRP ini terdapat beberapa asumsi, di antaranya:
1. Semua kendaraan yang digunakan memiliki jenis dan kapasitas yang sama 
2. CVRP tidak mempertimbangkan time window. Terdapat model kusus bernama VRPTW.
3. CVRP hanya mempertimbangkan kapasitas pengiriman.

## 1. Mengimpor Library dan Mengumpulkan Data

In [1]:
import numpy as np
from pulp import *
import gmaps
import googlemaps
import pandas as pd

In [2]:
#masukkan api key terlebih dahulu untuk membuat peta dan mengakses jarak antar tempat
#jika belum memiliki epi key, silakan mendaftar terlebih dahulu di https://developers.google.com/maps
API_KEY = 'Masukkan API Key di sini'
gmaps.configure(api_key=API_KEY)
googlemaps = googlemaps.Client(key=API_KEY)

In [86]:
#data saya ambil manual menggunakan gmaps
#data demand bersifat dummy
data = {"Nama":["Depot",
                "Alfamart Bugisan",
                "Alfamart Ring Road Selatan",
                "Alfamart Bibis",
                "Alfamart UMY Tamantirto",
                "Alfamart Ngebel",
                "Alfamart Ring Road Selatan 2"],
        "Lokasi":[(-7.820275519559818, 110.35576478748318),
                  (-7.818459939237757, 110.34828905480705),
                  (-7.826882740143658, 110.3457904293808),
                  (-7.825849387004239, 110.32770847947735),
                  (-7.814562872952285, 110.32839097560857),
                  (-7.814073787921593, 110.3183264772633),
                  (-7.809770036857429, 110.32472496772033)],
        "Demand":[0,40,50,50,30,20,10]}

#asumsi kendaraan dapat mengangkut 100 unit produk
vehicle_cap = 100
alfa = pd.DataFrame(data).set_index("Nama")
alfa

,Lokasi,Demand
Nama,,
Depot,"(-7.820275519559818, 110.35576478748318)",0
Alfamart Bugisan,"(-7.818459939237757, 110.34828905480705)",40
Alfamart Ring Road Selatan,"(-7.826882740143658, 110.3457904293808)",50
Alfamart Bibis,"(-7.825849387004239, 110.32770847947735)",50
Alfamart UMY Tamantirto,"(-7.814562872952285, 110.32839097560857)",30
Alfamart Ngebel,"(-7.814073787921593, 110.3183264772633)",20
Alfamart Ring Road Selatan 2,"(-7.809770036857429, 110.32472496772033)",10


## 2. Visualisasi Peta

In [87]:
#melihat lokasi 
alfa_map = gmaps.figure()

#untuk depot saya beri warna biru sebagai pembeda
depot = gmaps.symbol_layer([alfa.Lokasi[0]],fill_color="blue",stroke_opacity=0,
                           scale=6,info_box_content="Depot",display_info_box=True)

#tambahkan marker sebagai penanda retailer
markers = gmaps.marker_layer(alfa.Lokasi[1:])
 
alfa_map.add_layer(depot)    
alfa_map.add_layer(markers)
    
alfa_map

Figure(layout=FigureLayout(height='420px'))

Tampilan output akan seperti gambar di bawah ini:

<img src="https://user-images.githubusercontent.com/61647791/144628288-870c3c6a-ca83-4d02-a273-48b6cd4866be.png" />

## 3. Jarak Antar Lokasi
Tahap selanjutnya yakni menghitung jarak antar lokasi menggunakan api direction dari gmaps. Jarak yang digunakan merupakan jarak terdekat antar lokasi dengan mode "driving". Setiap jarak antar lokasi kemudian dimasukkan ke dalam sebuah matriks persegi dengan ukuran panjang dan lebar sesuai jumlah lokasi termasuk depot.  Berikut adalah kodenya:

In [88]:
#fungsi untuk membuat matriks jarak dari sebuah kolom lokasi

def distance_matrix(loc_column):
    #membuat matriks nol berukuran nxn
    distance_result = np.zeros((len(loc_column),len(loc_column)))
    
    for i in range(len(loc_column)):
        for j in range(len(loc_column)):
            
            # menghitung jarak antar lokasi
            api_result = googlemaps.directions(loc_column[i],
                                               loc_column[j],
                                               mode = 'driving')
            
            # memasukkan hasil perhitungan pada matriks 
            distance_result[i][j] = api_result[0]['legs'][0]['distance']['value']
    
    return distance_result

In [89]:
#jika perlu, matriks jarak dapat dibuat menjadi dataframe supaya lebih mudah dipahami
#jarak dalam satuan m

data_jarak = pd.DataFrame(distance_matrix(alfa.Lokasi),columns=alfa.index,index=alfa.index)
data_jarak

Nama,Depot,Alfamart Bugisan,Alfamart Ring Road Selatan,Alfamart Bibis,Alfamart UMY Tamantirto,Alfamart Ngebel,Alfamart Ring Road Selatan 2
Nama,,,,,,,
Depot,0.0,1349.0,1782.0,4264.0,4693.0,5764.0,5823.0
Alfamart Bugisan,1349.0,0.0,2479.0,3536.0,3002.0,5036.0,4132.0
Alfamart Ring Road Selatan,2518.0,1217.0,0.0,2482.0,2911.0,3982.0,4041.0
Alfamart Bibis,4269.0,3552.0,3155.0,0.0,1279.0,2350.0,2409.0
Alfamart UMY Tamantirto,4667.0,3002.0,4176.0,1280.0,0.0,1689.0,1130.0
Alfamart Ngebel,6246.0,5529.0,5755.0,2827.0,1695.0,0.0,3291.0
Alfamart Ring Road Selatan 2,5633.0,4916.0,5142.0,2245.0,1070.0,1732.0,0.0


In [ ]:
#teman-teman juga bisa menyimpan data ini terlebih dahulu
data_jarak.to_csv("jarak_alfamart.csv")

## 4. Pembuatan Model
Model VRP yang saya saya gunakan berasal dari Paolo Toth dan Daniele Vigo yang bisa dilihat melalui link paper di bawah ini:

https://www.sciencedirect.com/science/article/pii/S0166218X01003511

Terdapat dua buah variabel keputusan pada model ini, yaitu Xij yang bertipe binary dan menunjukkan apakah produk dikirimkan dari lokasi i ke j serta variabel jumlah rute yang digunakan. Misalkan rute yang diperlukan berjumlah 3 maka perusahaan bisa mengalokasikan 3 kendaraan untuk mengirimkan produk sekaligus, atau mengalokasikan 1 kendaraan untuk melakukan pengiriman sebanyak 3 kali. Berikut adalah model matematis yang digunakan:

<img src="https://user-images.githubusercontent.com/61647791/144783435-1bd7ccdf-441b-4401-a02b-857d4ccb6c5c.png" height=800 width=500 />


Paolo Toth dan Daniele Vigo juga memberikan batasan alternatif sebagai pengganti batasan **nomor 5** yang biasa disebut sebagai *generalized subtour elimination constraints (GSEC)*. Batasan ini jauh lebih mudah diterjemahkan ke dalam kode python.

<img src="https://user-images.githubusercontent.com/61647791/144783942-520c9ca2-49fb-40f5-b565-a90032238bfc.png" width=500 height=300/>

In [98]:
#inisiasi model
model = LpProblem("VRP",LpMinimize)

#decision variable
x_keys = [(i,j) for i in alfa.index for j in alfa.index]
x = LpVariable.dicts("x",x_keys,cat="Binary")

n_route = LpVariable("n_vehicle",cat="Integer") 

In [99]:
#fungsi tujuan: meminimalkan jarak
model += lpSum(data_jarak.loc[i,j]*x[i,j] for i,j in x_keys)

In [100]:
#constrain 0
#pastikan tidak ada arc dari lokasi awal dan tujuan yang sama, ex. alfamart bugisan ke alfamart bugisan
for i in alfa.index:
    model += x[i,i] == 0

In [101]:
#constrain 1: indegree constraint
#terdapat satu jalur selain dari depot yang masuk ke lokasi j
for j in alfa.iloc[1:].index:
    model += lpSum(x[i,j] for i in alfa.index) == 1

#constrain 2: outdegree constraint
#terdapat satu jalur selain dari depot yang keluar dari lokasi i
for i in alfa.iloc[1:].index:
    model += lpSum(x[i,j] for j in alfa.index) == 1

In [102]:
#constrain 3: terdapat n jalur yang keluar dari depot
model += lpSum(x["Depot",j] for j in alfa.index) == n_route

#constrain 4: terdapat n jalur yang masuk ke depot
model += lpSum(x[i,"Depot"] for i in alfa.index) == n_route

In [103]:
#constrain 5: subtour elimination
#list untuk menampung subtour
import itertools as it

subtours = []
for i in range(2,len(alfa.index)):
    subtours += it.combinations(alfa.index[1:], i)

for st in subtours:
    demand = np.sum([alfa.Demand[s] for s in st])
    model += lpSum(x[i,j] for i,j in it.permutations(st,2)) <= np.max([0,len(st)-np.ceil(demand/vehicle_cap)])

In [104]:
#fungsi untuk menemukan subtour, dan hasil route

def get_plan(r0):
    r=copy.copy(r0)
    route = []
    while len(r) != 0:
        plan = [r[0]]
        del (r[0])
        l = 0
        while len(plan) > l:
            l = len(plan)
            for i, j in enumerate(r):
                if plan[-1][1] == j[0]:
                    plan.append(j)
                    del (r[i])
        route.append(plan)
    return(route)

In [105]:
import copy
keys = [(i,j) for i in alfa.index for j in alfa.index]
status=model.solve()
#status=model.solver()
print("-----------------")
print(status,LpStatus[status],value(model.objective))
routes=[(i,j) for i,j in x_keys if value(x[i,j])==1]

-----------------
1 Optimal 21992.0


In [106]:
#mengecek rute
#terdapat dua buah rute
get_plan(routes)

[[('Depot', 'Alfamart Bugisan'),
  ('Alfamart Bugisan', 'Alfamart UMY Tamantirto'),
  ('Alfamart UMY Tamantirto', 'Alfamart Ring Road Selatan 2'),
  ('Alfamart Ring Road Selatan 2', 'Alfamart Ngebel'),
  ('Alfamart Ngebel', 'Depot'),
  ('Depot', 'Alfamart Ring Road Selatan'),
  ('Alfamart Ring Road Selatan', 'Alfamart Bibis'),
  ('Alfamart Bibis', 'Depot')]]

In [107]:
#memisahkan rute 
route_list = [[r1 for r1 in get_plan(routes)[0][:5]],[r2 for r2 in get_plan(routes)[0][5:]]]
route_list

[[('Depot', 'Alfamart Bugisan'),
  ('Alfamart Bugisan', 'Alfamart UMY Tamantirto'),
  ('Alfamart UMY Tamantirto', 'Alfamart Ring Road Selatan 2'),
  ('Alfamart Ring Road Selatan 2', 'Alfamart Ngebel'),
  ('Alfamart Ngebel', 'Depot')],
 [('Depot', 'Alfamart Ring Road Selatan'),
  ('Alfamart Ring Road Selatan', 'Alfamart Bibis'),
  ('Alfamart Bibis', 'Depot')]]

## 4. Visualisasi Rute

In [108]:
# visualisasi menggunakan gmaps
fig = gmaps.figure()
layer = []

#setiap rute memiliki warna yang berbeda
color_list = ["red","blue"]

#menggambar rute
for route,color in zip(route_list,color_list):
    for r in route:
        layer.append(gmaps.directions.Directions(alfa.loc[r[0]]["Lokasi"],
                                                 alfa.loc[r[1]]["Lokasi"],
                                                 mode="car",stroke_weight=5, stroke_color=color,
                                                 show_markers=False))

for i in range(len(layer)):
    fig.add_layer(layer[i])    
    
    
#menambahkan marker
markers = gmaps.marker_layer(alfa.Lokasi, info_box_content=alfa.index)
fig.add_layer(markers) 

fig

Figure(layout=FigureLayout(height='420px'))

Tampilan output akan seperti gambar di bawah ini:

<img src="https://user-images.githubusercontent.com/61647791/144784928-385aa154-cda4-4241-ac92-16c921e4d20a.png" />


<img src="https://user-images.githubusercontent.com/61647791/144784983-59f705aa-5927-44c0-b179-61c1b617f57b.png" />


In [44]:
#apabila ingin menyimpan peta
from ipywidgets.embed import embed_minimal_html

embed_minimal_html('alfa.html', views=[fig])


## Kesimpulan:

1. Berdasarkan hasil optimasi diperoleh total jarak yang ditempuh sebesar 21.992 km dengan dua rute/jalur. Perusahaan dapat mengalokasikan dua buah kendaraan untuk melakukan pengiriman dalam satu kali jalan atau mengalokasikan satu kendaraan untuk melakukan pengiriman sebanyak 2 kali.
2. Rute 1 merupakan pengiriman dari Depot -> Alfamart Bugisan -> Alfamart UMY Tamantirto -> Alfamart Ring Road Selatan 2 -> Alfamart Ngebel kemudian kembali ke Depot. Pada peta rute ini berwarna biru.
3. Rute 2 merupakan pengiriman dari Depot -> Alfamart Ring Road Selatan -> Alfamart Bibis kemudian kembali ke Depot. Pada peta rute ini berwarna merah.
4. Model CVRP ini hanya mempertimbangkan kapasitas produk yang dapat dikirimkan oleh kendaraan. Selain itu kendaaran diasumsikan homogen, dalam arti memiliki jenis serta kapasitas produk yang sama. Masih terdapat model VRP lainnya seperti pada bagan berikut:

<img src="https://www.researchgate.net/profile/Semih-Yalcindag/publication/267508322/figure/fig1/AS:295585207865357@1447484413863/Relationship-between-the-TSP-VRP-and-variants.png" />
